# this code ran on google colab. the execution time on a regular CPU will be very long

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sentence_transformers import SentenceTransformer
from imblearn.over_sampling import SMOTE

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import io
df = pd.read_csv(io.BytesIO(uploaded['data_trustpilot.csv']))

In [ ]:
df.head()

In [ ]:
df["rating"].value_counts()

In [ ]:
# Initialize the pre-trained sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')

# Extract the text data (preprocessed Text) and labels (ratings)
X = df['text_processed'].tolist()
y = df['rating'].tolist()

# Encode the text data into sentence embeddings
X_embeddings = model.encode(X)

# Apply SMOTE to oversample minority classes
smote = SMOTE()
texts, ratings = smote.fit_resample(X_embeddings, y)

# The result of resampling is now in `texts` and `ratings`

# new class distribution after resampling
print(pd.Series(ratings).value_counts())

import pickle
from google.colab import drive
drive.mount('/content/drive')


# Save the SMOTE output
with open('/content/drive/MyDrive/smote_output.pkl', 'wb') as f:
    pickle.dump([texts, ratings], f)

In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive')

# Open the file in read binary ('rb') mode
with open('/content/drive/MyDrive/smote_output.pkl', 'rb') as f:
    # Load the data using pickle.load()
    texts, ratings = pickle.load(f)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN
from sklearn.preprocessing import LabelEncoder
import numpy as np


texts = np.array(texts)  # Convert to NumPy array
texts = texts.reshape(texts.shape[0], 1, texts.shape[1])  # Reshape to (samples, 1, features)

# Explicitly convert the data type to float32
texts = texts.astype(np.float32)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, ratings, test_size=0.2, random_state=42)

# Create a LabelEncoder object
encoder = LabelEncoder()

# Fit the encoder on ratings data
encoder.fit(ratings)

# Transform the training and testing labels
y_train = encoder.transform(y_train)
y_test = encoder.transform(y_test)

# Ensure data type consistency
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# LSTM model
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(texts.shape[1], texts.shape[2])))  # Input shape matches embeddings
model.add(Dense(units=64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(units=5, activation='softmax'))  # 5 classes for 5-star ratings

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the Model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

# Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print("Validation Accuracy: ", accuracy * 100, "%")

# Save the model
model_save_path = '/content/drive/MyDrive/rnn_glove_rating_SMOTE_processed.h5'
model_save_path_1 = '/content/drive/MyDrive/rnn_glove_rating_SMOTE_processed.keras'

model.save(model_save_path)
model.save(model_save_path_1)

# plot training curve
import matplotlib.pyplot as plt
train_acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]

plt.plot(train_acc, label = "train_acc")
plt.plot(val_acc, label = "val_acc")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# print classification report and confusion matrix
y_pred = model.predict(X_test)

import numpy as np
test_pred_class= y_pred.argmax(axis = 1)
y_test_class = y_test

from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(y_test_class, test_pred_class))
print(confusion_matrix(y_test_class, test_pred_class))


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# 'texts' contains the SMOTE-generated embeddings
# 'X_embeddings' contains the original embeddings

# Initialize the pre-trained sentence transformer model
model = SentenceTransformer('all-mpnet-base-v2')



# Extract the text data (preprocessed Text) and labels (ratings)
X = df_2['text_processed'].tolist()
y = df_2['rating'].tolist()

# Encode the text data into sentence embeddings
X_embeddings = model.encode(X)

# Calculate cosine similarity between generated and original embeddings
similarity_matrix = cosine_similarity(texts, X_embeddings)

# For each generated embedding, find the index of the nearest neighbor
nearest_neighbors = similarity_matrix.argmax(axis=1)

# Retrieve the corresponding sentences from the original data
generated_sentences = df_2['text_processed'].iloc[nearest_neighbors].tolist()

# Print or inspect the generated sentences
for sentence in generated_sentences:
    print(sentence)
